<a href="https://colab.research.google.com/github/realtechsupport/cocktail/blob/main/sandbox/modularize_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Target preprocessing - clipping and preprocessing

In [ ]:
#imports and dowloads

!pip install rasterio
!pip install gdal

import os
import numpy as np
import rasterio
from osgeo import gdal, osr
import keras
import tensorflow as tf

In [ ]:
#target clipping

rasterfile = gdal.Open(datapath + ps)

print('\nPerforming the clip operation...\n')


warp_options = gdal.WarpOptions(cutlineDSName = roipath + roishape, cropToCutline = True)
rasterfile_new = ps.split('.tif')[0] + '_roi.tif'
print(roipath + rasterfile_new, datapath + ps )
ds = gdal.Warp(roipath + rasterfile_new, datapath + ps,  options = warp_options)

cols = ds.RasterXSize
rows = ds.RasterYSize
bands = ds.RasterCount
projInfo = ds.GetProjection()
spatialRef = osr.SpatialReference()
spatialRef.ImportFromWkt(projInfo)
spatialRefProj = spatialRef.ExportToProj4()
ds = None

print('\nClipped raster input: ', rasterfile_new)
print('Checking spatial reference info\n')
print ("WKT format: " + str(spatialRef))
print ("Proj4 format: " + str(spatialRefProj))
print ("Number of columns: " + str(cols))
print ("Number of rows: " + str(rows))
print ("Number of bands: " + str(bands))


datapath = '/content/gdrive/MyDrive/exp/'
ps = 'crop_mask.tif'
patch_size = 256
roipath = '/content/gdrive/MyDrive/exp/other images/roi_folder/'
roishape = 'area2_square.geojson'

#---------------------------------

#target preprocessing

def target_preprocessing(folder_path, patch_size):

    with rasterio.open(folder_path) as src:
        # Read the TIFF data
        output_mask = src.read()

        # Get the shape of the TIFF data
        num_bands, height, width = output_mask.shape

        # Calculate the new width and height that are multiples of the patch size
        new_width = int(np.floor(width / patch_size)) * patch_size
        new_height = int(np.floor(height / patch_size)) * patch_size

        print("cropped dimensions:", new_height, new_width)

        output_mask = np.moveaxis(output_mask, 0, -1)

        # Crop the input_image to the new dimensions
        cropped_mask = output_mask[:new_height, :new_width, :]

    new_mask = np.squeeze(cropped_mask)

    masks = []
    for i in range(0, new_mask.shape[0], patch_size):
        for j in range(0, new_mask.shape[1], patch_size):
            patch = new_mask[i:i+patch_size, j:j+patch_size]
            masks.append(patch)

    mask_array = np.array(masks)

    useful_masks = []
    useless = 0
    indexes = []
    for index,img in enumerate(mask_array):

        val, counts = np.unique(img, return_counts=True)

        if (1 - (counts[0]/counts.sum())) > 0.05:
          useful_masks.append(img)
          indexes.append(index)
        else:
          #print("I am useless")
          useless +=1


    print("Total useful images are: ", len(mask_array)-useless)
    print(indexes)
    print("Total useless images are: ", useless)

    useful_masks_array = np.array(useful_masks)

    useful_masks_onehot = to_categorical(useful_masks_array)


    return useful_masks, useful_masks_onehot, indexes


folder_path = '/content/gdrive/MyDrive/exp/other images/roi_folder/crop_mask_roi.tif'
patch_size = 256
target = target_preprocessing(folder_path,patch_size)

Image preprocessing:

- contains clipping, preprocessing, sampling and multi-image processing functions

In [ ]:
def newclipping(datapath,roipath,ps, roishape):
  rasterfile = gdal.Open(datapath + ps)

  print('\nPerforming the clip operation...\n')


  warp_options = gdal.WarpOptions(cutlineDSName = roipath + roishape, cropToCutline = True)
  rasterfile_new = ps.split('.tif')[0] + '_roi.tif'
  print(roipath + rasterfile_new, datapath + ps )
  ds = gdal.Warp(roipath + rasterfile_new, datapath + ps,  options = warp_options)

  cols = ds.RasterXSize
  rows = ds.RasterYSize
  bands = ds.RasterCount
  projInfo = ds.GetProjection()
  spatialRef = osr.SpatialReference()
  spatialRef.ImportFromWkt(projInfo)
  spatialRefProj = spatialRef.ExportToProj4()
  ds = None

  print('\nClipped raster input: ', rasterfile_new)
  print('Checking spatial reference info\n')
  print ("WKT format: " + str(spatialRef))
  print ("Proj4 format: " + str(spatialRefProj))
  print ("Number of columns: " + str(cols))
  print ("Number of rows: " + str(rows))
  print ("Number of bands: " + str(bands))

  return roipath + rasterfile_new

In [ ]:
def preprocessing(filelocation,patch_size):
    # Load the GeoTIFF file
    with rasterio.open(filelocation) as src:
        # Read the TIFF data
        tiff_data = src.read()
        print("total number of nan in original",np.count_nonzero(np.isnan(tiff_data)))

        # Get the shape of the TIFF data
        num_bands, height, width = tiff_data.shape

        print("Original image dimensions:", num_bands, height, width)
        unique_elements, counts_elements = np.unique(tiff_data, return_counts=True)
        print(unique_elements, counts_elements )
        print("total unique",len(counts_elements))

        print(np.min(tiff_data), np.max(tiff_data))


        normalized_image = np.zeros_like(tiff_data, dtype='float32')

        for band, count in enumerate(range(tiff_data.shape[0])):
            band_data = tiff_data[band, :, :]
            band_min = np.min(band_data)
            band_max = np.max(band_data)
            print("band-", count+1,"maximum-",band_max,"minimum-",band_min)
            #print(band_data)
            #epsilon is added to prevent division by zero
            normalized_band = (band_data - band_min) / (band_max - band_min + 1e-10)
            normalized_image[band, :, :] = normalized_band

        # Calculate the new width and height that are multiples of the patch size
        new_width = int(np.floor(width / patch_size)) * patch_size
        new_height = int(np.floor(height / patch_size)) * patch_size

        print("cropped dimensions:", new_height, new_width)

        input_image = np.moveaxis(normalized_image, 0, -1)

        # Crop the input_image to the new dimensions
        cropped_array = input_image[:new_height, :new_width, :]

    print("total number of nan",np.count_nonzero(np.isnan(cropped_array)))
    print("Cropped array shape:", cropped_array.shape)
    print(np.min(cropped_array), np.max(cropped_array))

    patches = []
    for i in range(0, cropped_array.shape[0], patch_size):
        for j in range(0, cropped_array.shape[1], patch_size):
            patch = cropped_array[i:i+patch_size, j:j+patch_size]
            patches.append(patch)
    print("patches are created")
    return patches

In [ ]:
def sampling(patches,indexes):
    useful_patches = []
    for number_of_masks,i in enumerate(indexes):
      print(i)
      if i < len(patches):
        useful_patches.append(patches[i])
      else:
        break
    return useful_patches, number_of_masks



In [ ]:
def process_images_in_folder(datapath, patch_size, roipath,target, roishape = 'area2_square.geojson'):
    # Initialize an empty list to store the sampled patches and target_patches
    sampled_image_patches = []
    target_patches = []

    # Loop through all files in the folder
    for ps in os.listdir(datapath):

        # #print(type(filename),filename.split('.tif')[0] + '_roi.tif' )
        # print(folder_path + ps)
        # print(roipath + roishape)
        # print(ps)

        # Apply clipping
        clipped_image_path = newclipping(datapath,roipath,ps, roishape)

        # Preprocess(normalize,resize,create patches) the clipped image
        patches = preprocessing(clipped_image_path, patch_size)
        print(len(patches))

        # Sample patches from the processed image patches and create corresponding target patches
        #print(target[2])
        #print(len(target[2]))
        useful_patches, number_of_masks = sampling(patches,target[2])
        sampled_image_patches.extend(useful_patches)
        target_patches.extend(target[1][0:number_of_masks])
        print(len(sampled_image_patches),len(target_patches) )


    return sampled_image_patches,target_patches

In [ ]:
datapath = '/content/gdrive/MyDrive/exp/other images/test_folder/'
roipath = '/content/gdrive/MyDrive/exp/other images/roi_folder/'
patch_size = 256

dataset = process_images_in_folder(datapath, patch_size, roipath, target)

Training:
1. load the model
2. create the training and val sets and train the model

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (8,))
    x = inputs

    # Encoder
    conv1 = layers.Conv2D(16, 3, activation="relu", padding="same")(x)
    conv1 = layers.Conv2D(16, 3, activation="relu", padding="same")(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(32, 3, activation="relu", padding="same")(pool1)
    conv2 = layers.Conv2D(32, 3, activation="relu", padding="same")(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    # Bottleneck
    conv3 = layers.Conv2D(64, 3, activation="relu", padding="same")(pool2)
    conv3 = layers.Conv2D(64, 3, activation="relu", padding="same")(conv3)

    # Decoder
    up4 = layers.UpSampling2D(size=(2, 2))(conv3)
    concat4 = layers.Concatenate()([up4, conv2])
    conv4 = layers.Conv2D(32, 3, activation="relu", padding="same")(concat4)
    conv4 = layers.Conv2D(32, 3, activation="relu", padding="same")(conv4)

    up5 = layers.UpSampling2D(size=(2, 2))(conv4)
    concat5 = layers.Concatenate()([up5, conv1])
    conv5 = layers.Conv2D(16, 3, activation="relu", padding="same")(concat5)
    conv5 = layers.Conv2D(16, 3, activation="relu", padding="same")(conv5)

    # Output
    outputs = layers.Conv2D(num_classes, 1, activation="softmax")(conv5)

    model = keras.Model(inputs, outputs)
    return model

img_size = (256, 256)
num_classes = 21
model = get_model(img_size=img_size, num_classes=num_classes)
model.summary()

In [ ]:
import numpy as np
import random

img_size = (256, 256)
num_imgs = len(dataset[0])

input_img_paths = dataset[0]
target_paths = dataset[1]

random.Random(1337).shuffle(input_img_paths)
random.Random(1337).shuffle(target_paths)


input_imgs = np.zeros((num_imgs,) + img_size + (8,), dtype="float32")
targets = np.zeros((num_imgs,) + img_size + (21,), dtype="uint8")

for i in range(num_imgs):

    input_imgs[i] = input_img_paths[i]
    targets[i] = target_paths[i]

num_val_samples = round(0.2 * num_imgs)

train_input_imgs = input_imgs[:-num_val_samples]
train_targets = targets[:-num_val_samples]

val_input_imgs = input_imgs[-num_val_samples:]
val_targets = targets[-num_val_samples:]



In [ ]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

callbacks = [
    keras.callbacks.ModelCheckpoint("new_segmentation.hdf5",
                                    save_best_only=True)
]

history = model.fit(train_input_imgs, train_targets,
                    epochs=5,
                    callbacks=callbacks,
                    batch_size=8,
                    validation_data=(val_input_imgs, val_targets))

In [ ]:
epochs = range(1, len(history.history["loss"]) + 1)
loss = history.history["loss"]
val_loss = history.history["val_loss"]
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()

Testing prediction:
Same from old code